In [ ]:
import os
import sys

sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"
# os.environ["NUMBA_DEBUG_FRONTEND"] = "1"
# os.environ["NUMBA_DEBUG"] = "1"
# os.environ["NUMBA_DEBUGINFO"] = "1"
# os.environ["NUMBA_SHOW_HELP"] = "1"
# os.environ["NUMBA_FULL_TRACEBACKS"] = "1"
# os.environ["NUMBA_DEVELOPER_MODE"] = "1"
os.environ["NUMBA_DEBUG_CACHE"] = "1"
# os.environ.get("NUMBA_DEBUG_CACHE")

In [ ]:
import numba
from my_stuff import MufexKeys
from nb_quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from nb_quantfreedom.nb_base import backtest_df_only
from nb_quantfreedom.nb_enums import *
from nb_quantfreedom.nb_helper_funcs import nb_dos_cart_product
from nb_quantfreedom.strategies.nb_strategy import nb_BacktestInd, nb_Strategy
from numba.experimental import jitclass
from nb_quantfreedom.nb_custom_logger import nb_PrintLogs
from nb_quantfreedom.nb_order_handler.nb_class_helpers import nb_GetMinPrice
from nb_quantfreedom.utils import clear_cache


clear_cache()

mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2


In [ ]:
candles = mufex_main.get_candles_df(
    symbol="BTCUSDT",
    timeframe="15m",
    candles_to_dl=200,
)

In [ ]:
@jitclass([("svoxel", numba.types.int64[:])])
class FeedBuilder(object):
    def __init__(self, svoxel):
        self.svoxel = svoxel

    def hello(self):
        print("hello world")

    def iterate(self, callback):
        callback(self)


@numba.njit(cache=True)
def callbackFunc(feedBuilder):
    feedBuilder.hello()
    print("callbackFunc")
    # print(feedBuilder.svoxel[1])
    # feedBuilder.svoxel[0] = 900
    # print(feedBuilder.svoxel[0])


@numba.njit(cache=True)
def wrapper():
    fBuilder = FeedBuilder(np.array([1, 10], np.int64))
    fBuilder.iterate(callbackFunc)


wrapper()

In [ ]:
class nb_GlobalVars:
    def __init__(
        self,
        price_tick_step,
        market_fee_pct,
    ):
        self.price_tick_step = price_tick_step
        self.market_fee_pct = market_fee_pct


global_vars_list = [
    ("price_tick_step", numba.types.int_),
    ("market_fee_pct", numba.types.float_),
]


class TestSLClass:
    def __init__(self):
        pass

    def calc_sl_callback(self, callback):
        pass

    def calc_sl(self, price, lookback, thinger):
        pass


@jitclass(global_vars_list)
class SLCalc(nb_GlobalVars, TestSLClass):
    def calc_sl_callback(self, callback):
        return callback(self)

    def calc_sl(self, price, lookback, thinger):
        return self.price_tick_step * self.market_fee_pct * price - thinger / lookback


@jitclass(global_vars_list)
class newSLCals(nb_GlobalVars, TestSLClass):
    def calc_sl_callback(self, callback):
        return callback(self)

    def calc_sl(self, price, lookback, thinger):
        return self.price_tick_step * self.market_fee_pct * price / lookback / thinger


@numba.njit(cache=True)
def nb_simulate(sl_calculator: TestSLClass):
    return sl_calculator.calc_sl(price=10, lookback=5, thinger=6)


@numba.njit(cache=True)
def nb_base():
    sl_calculator = SLCalc(1, 10.5)
    return sl_calculator.calc_sl_callback(nb_simulate)

nb_base()

In [ ]:
# @numba.njit(cache=True)
def composition(funcs, x):
    r = x
    for f in funcs[::-1]:
        print(numba.typeof(f))
        r = f(r)
    return r


@numba.cfunc("float64(float64)", cache=True)
def a(x):
    return x + 1.0


@numba.njit(cache=True)
def b(x):
    return x * x


composition((a, b), 0.5), 2

In [ ]:
import numba
from numba.core import types

@numba.njit()
def f1(x, y):
    return x + y

@numba.njit()
def f2(x, y):
    return x + y

f1(1., 2.)
f2(1., 2.)

f_list = numba.typed.List.empty_list(types.float64(types.float64, types.float64).as_type())
f_list.append(f1)
f_list.append(f2)

@numba.experimental.jitclass([('funcs', types.ListType(types.float64(types.float64, types.float64).as_type()))])
class Handler:

    def __init__(self, funcs):
        self.funcs = funcs

In [ ]:
Handler.class_type

In [ ]:
import numba
from numba.core import types


class PClass:
    def __init__(self) -> None:
        pass


@numba.experimental.jitclass()
class TestingTEster(PClass):
    def f1(x, y):
        return x + y

    def f2(x, y):
        return x + y


@numba.experimental.jitclass(
    [("funcs", types.ListType(TestingTEster.class_type.instance_type(types.float64(types.float_, types.float_))))]
)
class JITTEST:
    def __init__(self, funcs):
        self.funcs = funcs

In [ ]:
# https://numba.discourse.group/t/typed-list-of-jitted-functions-in-jitclass/413/6
import logging
import numba
from numba.core import types


@numba.njit(cache=True)
def log_debug(message):
    print(message+ " this is log debug")
    pass


@numba.njit(cache=True)
def log_info(message):
    print(message + " this is log info")
    pass


@numba.experimental.jitclass(
    [
        ("log_debug", types.void(types.unicode_type).as_type()),
        ("log_info", types.void(types.unicode_type).as_type()),
    ]
)
class PrintTester:
    def __init__(self, log_debug, log_info):
        if log_debug is not None:
            self.log_debug = log_debug
        elif log_debug is None:
            self.log_debug = logging.debug
        self.log_info = log_info


logger = PrintTester(None, log_info)

In [ ]:
class asdfasdf(object):
    def __init__(self, hey,) -> None:
        self.hey = hey

class Wiwer(asdfasdf):
    def printer(self):
        print("hello")
        

asdfasdf(hey='youou')
Wiwer(hey="yes").printer()

In [ ]:
@numba.njit(cache=True)
def composition(funcs, x):
    r = x
    for f in range(10):
        r = funcs(f)
    return r


@numba.cfunc("float64(float64)", cache=True)
def a(x):
    return x + 1.0



composition(a, 0.5)

In [ ]:
@numba.njit(cache=True)
def nb_min_max_price_getter(
    self,
    logger_debug: numba.types.FunctionType,
    bar_index: int,
    candles: np.array,
    candle_body_type: int,
    lookback: int,
) -> float:
    price = candles[lookback : bar_index + 1 :, candle_body_type].min()
    logger_debug(
        "nb_class_helpers.py - nb_GetMinPrice - nb_min_max_price_getter() -"
        + " candle_body_type= "
        + logger.candle_body_str(candle_body_type)
        + " price= "
        + logger.float_to_str(price)
    )
    return price

In [ ]:
# https://numba.discourse.group/t/typed-list-of-jitted-functions-in-jitclass/413/6

from numba.core import types
from numba import njit
from numba.experimental import jitclass


@njit(cache=True)
def nb_remander(nb_func_num: float):
    return nb_func_num % 3.0


@njit(cache=True)
def nb_power(nb_func_num: float):
    return nb_func_num**2.0


# class HelperClass:
#     def __init__(self, helper_class_self_num: float, helper_class_self_var: float) -> None:
#         self.helper_class_self_num = helper_class_self_num
#         self.helper_class_self_var = helper_class_self_var

#     def calc_num_one(self, helper_num: float):
#         pass

#     def calc_num_two(self, helper_num: float):
#         pass


@jitclass([("helper_class_num", types.float_), ("helper_class_var", types.float_)])
class HelperClass(object):
    def __init__(self, helper_class_self_num: float, helper_class_self_var: float) -> None:
        self.helper_class_self_num = helper_class_self_num
        self.helper_class_self_var = helper_class_self_var

    def calc_num_one(self, helper_num: float):
        pass

    def calc_num_two(self, helper_num: float):
        pass


@jitclass([("helper_class_num", types.float_), ("helper_class_var", types.float_)])
class SubClassOne(object):
    helper_class_inst: HelperClass

    def __init__(self, helper_class_data) -> None:
        self.helper_class_inst = helper_class_data

    def calc_num_one(self, helper_num: float):
        return helper_num / self.helper_class_inst.helper_class_self_num

    def calc_num_two(self, helper_num: float):
        return helper_num - self.helper_class_inst.helper_class_self_var


@numba.experimental.jitclass([("helper_class_num", types.float_), ("helper_class_var", types.float_)])
class ClassTwo(object):
    helper_class_inst: HelperClass

    def __init__(self, helper_class_data) -> None:
        self.helper_class_inst = helper_class_data

    def calc_num_one(self, helper_num: float):
        return helper_num / self.helper_class_inst.helper_class_self_num

    def calc_num_two(self, helper_num: float):
        return helper_num - self.helper_class_inst.helper_class_self_var



In [ ]:

# @numba.experimental.jitclass(
#     [
#         # ("helper_class", nb_HelperClass.class_type.instance_type),
#         ("nb_func", types.float_(types.float_).as_type()),
#         ("jit_test_num", types.float_),
#     ]
# )
# class JitTest(object):
#     helper_class_inst: HelperClass

#     def __init__(self, helper_class_inst, nb_func, jit_test_num):
#         self.helper_class_inst = helper_class_inst
#         self.nb_func = nb_func
#         self.jit_test_num = jit_test_num

#     def final_calc(self, helper_num: float, nb_func_num: float):
#         return (
#             self.helper_class_inst.calc_num_two(helper_num=helper_num)
#             + self.nb_func(nb_func_num=nb_func_num)
#             + self.jit_test_num
#         )


# @njit(cache=True)
# def exec_nb_class():
#     do_jit_test = JitTest(
#         ClassTwo(helper_class_self_num=10.0, helper_class_self_var=20.0),
#         nb_power,
#         6.0,
#     )
#     return do_jit_test.final_calc(helper_num=30.0, nb_func_num=2.0)


# exec_nb_class()
"""
do_jit_test.final_calc(helper_num = 30.0, nb_func_num = 3.0)
    self.helper_class.calc_num_two(helper_num = 30.0) + self.nb_func(nb_func_num = 2) + self.jit_test_num
        self.helper_class.calc_num_two(helper_num = 30.0)
            ClassTwo().calc_num_two(helper_num = 30.0)
                helper_num - self.helper_class_self_var
                30.0 - 20.0 = 10.0

        self.nb_func(nb_func_num = 3.0)
            nb_power(nb_func_num = 3.0)
            3.0**2.0 = 4.0

        self.class_num = 6.0

        10.0 + 4.0 + 6.0 = 20.0

        do_jit_test.final_calc(helper_num = 30.0, nb_func_num = 2.0) = 20.0

exec_nb_class() = 20.0

"""

In [4]:
import logging
class reglogs(logging):
    def __init__(self):
        pass

logger = reglogs()

TypeError: module() takes at most 2 arguments (3 given)

In [ ]:
logging.debug()